# Simple Toy TS Training for Deep3D+

In [1]:
"""
Simple tester for the deep3d
"""
import tensorflow as tf
import Deep3D as deep3d
import utils
import numpy as np
import os
import os.path
import h5py
import matplotlib as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Importing Data from H5 Format for fast loading
- Will eventually unit test dynamic CPU data loading pipeline here

In [2]:
#inria_file = '/a/data/deep3d_data/inria_data.h5'
inria_file = 'data/inria_data.h5'
h5f = h5py.File(inria_file,'r')

X_train = h5f['X_0'][:,10:170,16:304,:]
Y_train = h5f['Y_0'][:,10:170,16:304,:]
# X_train_1 = h5f['X_1'][:,10:170,16:304,:]
# Y_train_1 = h5f['Y_1'][:,10:170,16:304,:]
# X_train_2 = h5f['X_2'][:,10:170,16:304,:]
# Y_train_2 = h5f['Y_2'][:,10:170,16:304,:]
# X_train_3 = h5f['X_3'][:,10:170,16:304,:]
# Y_train_3 = h5f['Y_3'][:,10:170,16:304,:]

X_val = h5f['X_4'][:,10:170,16:304,:]
Y_val = h5f['Y_4'][:,10:170,16:304,:]
  
h5f.close()

In [3]:
# X_train = np.concatenate([X_train_0,X_train_1,X_train_2,X_train_3])
# Y_train = np.concatenate([Y_train_0,Y_train_1,Y_train_2,Y_train_3])

print "Training Size:" + str(X_train.shape)
print "Validation Size:" + str(X_val.shape)

Training Size:(500, 160, 288, 3)
Validation Size:(500, 160, 288, 3)


## Training Loop

In [4]:
num_batches = 1
batchsize = 6
print_step = 1
#cost_hist = []
viz_step = 10

# Define config for GPU memory debugging 
config = tf.ConfigProto()
config.gpu_options.allow_growth=True  # Switch to True for dynamic memory allocation instead of TF hogging BS
config.gpu_options.per_process_gpu_memory_fraction= 1  # Cap TF mem usage

#with tf.device('/gpu:0'):
with tf.device('/cpu:0'):  
    # Session
    #sess = tf.Session(config=config)
    sess = tf.Session()
    
    # Placeholders
    images = tf.placeholder(tf.float32, [batchsize, 160, 288, 3], name='input_batch')
    true_out = tf.placeholder(tf.float32, [batchsize, 160, 288, 3] , name='ground_truth')
    train_mode = tf.placeholder(tf.bool, name='train_mode')

    # Building Net based on VGG weights 
    net = deep3d.Deep3Dnet('./vgg19.npy')
    net.build(images, train_mode)

    # Print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print 'Variable count:'
    print(net.get_var_count())
    
    # Run initializer 
    sess.run(tf.global_variables_initializer())
    
    # Define Training Objectives
    cost = tf.reduce_sum(tf.abs(net.prob - true_out), name='L1_loss')
    train = tf.train.GradientDescentOptimizer(0.002).minimize(cost)
    
    # tensorboard operations to compile summary and then write into logs
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tensorboard_logs/', graph = sess.graph)

    # Training Loop
    print ""
    print "== Start training =="
    for i in xrange(num_batches):
        # Creating Batch
        image_mask = np.random.choice(X_train.shape[0],batchsize)
        images_in = X_train[image_mask,:,:,:]
        labels_in = Y_train[image_mask,:,:,:]
        
        # Traing Step
        _, cost_val, summary = sess.run([train, cost, merged], feed_dict={images: images_in, true_out: labels_in, train_mode: True})
        writer.add_summary(summary, i)

        # No longer needed: cost_hist.append(cost_val)
        if i%print_step == 0:
            print ("({}/{})".format(i, num_batches).ljust(10) + ' | Cost: ' + str(cost_val))
        
    print ""
    print "Training Completed, storing weights"
    # Store Traing Output
    net.save_npy(sess)

Variable count:
138408071

== Start training ==
(0/1)      | Cost: 60336.9

Training Completed, storing weights
('file saved', './deep3d-save.npy')


## Testing Output

In [ ]:
del net

# Test
test_img = np.expand_dims(X_val[100], axis = 0)
test_ans = Y_val[100]


config = tf.ConfigProto()
config.gpu_options.allow_growth=False
config.gpu_options.per_process_gpu_memory_fraction= 1

with tf.device("/gpu:0"):
    sess = tf.Session(config=config)
    images = tf.placeholder(tf.float32, [1, 160, 288, 3])
    train_mode = tf.placeholder(tf.bool)
    
    net = deep3d.Deep3Dnet('./deep3d-save.npy')
    net.build(images, train_mode)
    sess.run(tf.global_variables_initializer())
    res, mask, up_3 = sess.run([net.prob, net.mask, net.up_3], feed_dict={images: test_img, train_mode: False})

In [ ]:
from matplotlib import pyplot
pyplot.figure()

print "--- Input ---"
pyplot.imshow(test_img[0])
pyplot.show()

print "--- GT ---"
pyplot.imshow(test_ans)
pyplot.show()

print "--- Our result ---"
pyplot.imshow(res[0])
pyplot.show()

#pyplot.imsave('1.jpeg', test_img[0])
#pyplot.imsave('2.jpeg', res[0])

## Looking at Disparity Maps

In [ ]:
f, axs = pyplot.subplots(6, 6, sharex='col', sharey='row')

for i in range(33):
    axs[i/6][i%6].imshow(mask[0,:,:,i],cmap="gray",vmin=0.0, vmax=1.0)
pyplot.show()

In [ ]:
max_shift_channel = np.argmax(mask,axis = 3)
max_shift_channel[0]